# [1164. Product Price at a Given Date](https://leetcode.com/problems/product-price-at-a-given-date/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Products

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| new_price     | int     |
| change_date   | date    |
+---------------+---------+</pre>
(product_id, change_date) is the primary key (combination of columns with unique values) of this table.
Each row of this table indicates that the price of some product was changed to a new price at some date.


Write a solution to find the prices of all products on 2019-08-16. Assume the price of all products before any change is 10.

Return the result table in any order.

The result format is in the following example.



Example 1:

Input:
Products table:
<pre>+------------+-----------+-------------+
| product_id | new_price | change_date |
+------------+-----------+-------------+
| 1          | 20        | 2019-08-14  |
| 2          | 50        | 2019-08-14  |
| 1          | 30        | 2019-08-15  |
| 1          | 35        | 2019-08-16  |
| 2          | 65        | 2019-08-17  |
| 3          | 20        | 2019-08-18  |
+------------+-----------+-------------+</pre>
Output:
<pre>+------------+-------+
| product_id | price |
+------------+-------+
| 2          | 50    |
| 1          | 35    |
| 3          | 10    |
+------------+-------+</pre>

In [3]:
# Pandas Schema
import pandas as pd

data = [[1, 20, '2019-08-14'], [2, 50, '2019-08-14'], [1, 30, '2019-08-15'], [1, 35, '2019-08-16'],
        [2, 65, '2019-08-17'], [3, 20, '2019-08-18']]

# data = [ #test case 4 from leet code
#     [1, 20, '2019-08-17'],
#     [2, 50, '2019-08-18'],
#     [1, 30, '2019-08-15'],
#     [1, 35, '2019-08-16'],
#     [2, 65, '2019-08-17'],
#     [3, 20, '2019-08-18']
# ]

products = pd.DataFrame(data, columns=['product_id', 'new_price', 'change_date']).astype(
    {'product_id': 'Int64', 'new_price': 'Int64', 'change_date': 'datetime64[ns]'})


# to spark schema

from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

products_df = spark.createDataFrame(products)
products_df.show(truncate=False)

+----------+---------+-------------------+
|product_id|new_price|change_date        |
+----------+---------+-------------------+
|1         |20       |2019-08-14 00:00:00|
|2         |50       |2019-08-14 00:00:00|
|1         |30       |2019-08-15 00:00:00|
|1         |35       |2019-08-16 00:00:00|
|2         |65       |2019-08-17 00:00:00|
|3         |20       |2019-08-18 00:00:00|
+----------+---------+-------------------+



In [61]:
#In pyspark dataframe

subdf = products_df\
             .where(col('change_date') <= '2019-08-16')\
             .select("product_id").distinct()

df1 = products_df\
    .join(subdf, on="product_id", how="left_anti")\
    .select("product_id").distinct()\
    .withColumn("price", lit(10))


subdf=products_df\
        .where(col('change_date') <= '2019-08-16')\
        .groupBy('product_id').agg(max('change_date').alias('mx'))

df2=products_df\
        .join(subdf.alias('b'), (col('change_date')==col('mx')) & (products_df.product_id==subdf.product_id), 'inner')\
        .select(('b.product_id'),col('new_price').alias('price'))


df1.union(df2).show()


# #note
# error you're encountering when using products_df['product_id'] or products_df.product_id in the select statement
# is due to the way PySpark handles column references, particularly in joins.


+----------+-----+
|product_id|price|
+----------+-----+
|         3|   10|
|         2|   50|
|         1|   35|
+----------+-----+



In [ ]:
#In pyspark dataframe using subquery

subdf = products_df\
             .where(col('change_date') <= '2019-08-16')\
             .select("product_id").distinct()
#Collecting in list
valid_combinations = [row.product_id for row in subdf.collect()]


#compare using isin
df1 = products_df\
    .where( ~col('product_id').isin(valid_combinations))\
    .select("product_id").distinct()\
    .withColumn("price", lit(10))


subdf=products_df\
        .where(col('change_date') <= '2019-08-16')\
        .groupBy('product_id').agg(max('change_date').alias('mx'))

# Collect (product_id, max_change_date) into separate lists
valid_product_ids = [row.product_id for row in subdf.collect()]
valid_change_dates = [row.mx for row in subdf.collect()]

# Filter using isin for product_id and change_date separately
dfx = products_df\
    .where(col('product_id').isin(valid_product_ids) & col('change_date').isin(valid_change_dates))\
    .select(('product_id'),col('new_price').alias('price'))

df1.union(dfx).show()


In [4]:
# In spark SQL

products_df.createOrReplaceTempView('products')

spark.sql('''
SELECT DISTINCT product_id, 10 AS price
FROM Products
WHERE product_id NOT IN (
    SELECT DISTINCT product_id
    FROM Products
    WHERE change_date <= '2019-08-16'
)
UNION
SELECT product_id, new_price AS price
FROM Products
WHERE (product_id, change_date) IN (
    SELECT product_id, MAX(change_date)
    FROM Products
    WHERE change_date <= '2019-08-16'
    GROUP BY product_id
)
''').show()

+----------+-----+
|product_id|price|
+----------+-----+
|         3|   10|
|         2|   50|
|         1|   35|
+----------+-----+



# TAKE AWAY
## EXAMPLE SHOWING FAILURE OF
## `products_df['product_id'] or products_df.product_id`

seeing some wiered behaviour while selecting
The error you're encountering when using products_df['product_id'] or products_df.product_id in the select statement is due to the way PySpark handles column references, particularly in joins.

In [68]:
#In pyspark dataframe unkown error INGNORE THIS
subdf = products_df\
             .where(col('change_date') <= '2019-08-16')\
             .select("product_id").distinct()

df1 = products_df\
    .join(subdf, on="product_id", how="left_anti")\
    .select("product_id").distinct()\
    .withColumn("price", lit(10))

df1.show()


subdf=products_df\
        .where(col('change_date') <= '2019-08-16')\
        .groupBy('product_id').agg(max('change_date').alias('mx'))

df2=products_df\
        .join(subdf, (col('change_date')==col('mx')) & (products_df.product_id==subdf.product_id), 'inner')\
        .select(products_df['product_id'],col('new_price').alias('price'))
df2.show()


df2=products_df\
        .join(subdf, (col('change_date')==col('mx')) & (products_df.product_id==subdf.product_id), 'inner')\
        .select(products_df.product_id,col('new_price').alias('price'))
df2.show()



"""
The error you're encountering when using products_df['product_id'] or products_df.product_id in the select statement
is due to the way PySpark handles column references, particularly in joins.
"""

+----------+-----+
|product_id|price|
+----------+-----+
|         3|   10|
+----------+-----+



AnalysisException: Column product_id#0L are ambiguous. It's probably because you joined several Datasets together, and some of these Datasets are the same. This column points to one of the Datasets but Spark is unable to figure out which one. Please alias the Datasets with different names via `Dataset.as` before joining them, and specify the column using qualified name, e.g. `df.as("a").join(df.as("b"), $"a.id" > $"b.id")`. You can also set spark.sql.analyzer.failAmbiguousSelfJoin to false to disable this check.

In [ ]:
spark.stop()